In [1]:
%load_ext autoreload
%autoreload 2
from utils import feature_eng, visualization
# plot the reward function with plotly as line plot
import plotly.io as pio
import numpy as np
import plotly.graph_objs as go
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import pandas as pd

from utils import eval

In [17]:
pio.renderers.default = 'iframe'

measurements = feature_eng.get_data_df("train", "B")
fig = visualization.plot_substation_measurements(measurements, "B")
fig.show()

In [3]:
fig = visualization.plot_production(measurements, ['ZLQZLQSPHSDL_VNUUIST'])
fig.show()

In [10]:
is_data = measurements['ZLQZLQSPHSDL_VNUUIST']
target_data = measurements['ZLQZLQSPHSDL_VNUUSOL']
shunt_reactor = measurements['ZLQZLQ110KOMDROQ']
station_b_val = feature_eng.add_temporal_features(feature_eng.get_data_df("val", "B"))

cut_off = 4 * 24 * 8 + 10
reactor_border = 20

train, test = train_test_split(is_data[:-cut_off].values, train_size=len(is_data[:-cut_off])-1)

model = pm.ARIMA(order=(2, 1, 2), seasonal_order=(1, 0, 1, 4))
print("fertig")


fertig


In [15]:
# measurement points are separated by 15 minutes. 4 to get to one hour.
time = 4
loops = 1

predictions = []
for i in range(loops):
    train, test = train_test_split(is_data[:-cut_off-i].values, train_size=len(is_data[:-cut_off-i])-1)
    print(len(train))
    print(len(pd.DataFrame(shunt_reactor[:-cut_off-i-1])))
    model_fit = model.fit(y=train, X=pd.DataFrame(shunt_reactor[:-cut_off-i-1]))
print("fertig")

34162
34162


C:\_dev\edh2023\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



fertig


In [ ]:
for i in range(len(station_b_val)-1):
    # Simulate with shunt reactor running
    preds_reactor_on = model_fit.predict(time, pd.DataFrame(np.full(time, 90)))

    # Simulate with shunt reactor off
    preds_reactor_off = model_fit.predict(time, pd.DataFrame(np.full(time, 0)))
    predictions.append(preds_reactor_on)
    predictions.append(preds_reactor_off)
    station_b_val['ZLQZLQ110KOMDROQ'][i].

In [16]:
predicted_datas = []

for i in range(loops):
    last_element = is_data.index[len(is_data)-cut_off-i]
    predicted_data = pd.Series(predictions[i*2], index=pd.date_range(last_element, periods=(time), freq='15T'))
    predicted_datas.append(predicted_data)
    predicted_data = pd.Series(predictions[i*2+1], index=pd.date_range(last_element, periods=(time), freq='15T'))
    predicted_datas.append(predicted_data)

print(is_data)
print(predicted_data)

is_reactor_running = shunt_reactor[:-cut_off] > 20



# calculate reward points achieved for the validation dataset
reward_base = eval.reward_sum(
    station_b_val["ZLQZLQSPHSDL_VNUUIST"].values - station_b_val["ZLQZLQSPHSDL_VNUUSOL"].values, 
    (station_b_val['ZLQZLQ110KOMDROQ'] > 1.0).values, 
    weighing_factors=np.ones(station_b_val.shape[0]))
print(f'Reward achieved by operating the shunt reactor as in the validation dataset: {reward_base}.')


if (

scatters = []
scatters.append(go.Scatter(x=is_data.index, y=is_data, name="IST"))

for i in range(loops):
    scatters.append(go.Scatter(x=predicted_datas[i*2].index, y=predicted_datas[i*2], name="PREDICT ON "+str(i)))
    scatters.append(go.Scatter(x=predicted_datas[i*2+1].index, y=predicted_datas[i*2+1], name="PREDICT OFF "+str(i)))

fig = go.Figure(data=scatters)

# Update the layout
fig.update_layout(title="Nice", xaxis_title='Date', yaxis_title='kV')
fig.show()

2022-01-01 01:00:00    239.816895
2022-01-01 01:15:00    239.816895
2022-01-01 01:30:00    239.868171
2022-01-01 01:45:00    240.080566
2022-01-01 02:00:00    240.080566
                          ...    
2022-12-30 23:00:00    239.553207
2022-12-30 23:15:00    240.323946
2022-12-30 23:30:00    240.476089
2022-12-30 23:45:00    240.476089
2022-12-31 00:00:00    240.476089
Freq: 15T, Name: ZLQZLQSPHSDL_VNUUIST, Length: 34941, dtype: float64
2022-12-22 21:45:00    239.138830
2022-12-22 22:00:00    239.116199
2022-12-22 22:15:00    239.091907
2022-12-22 22:30:00    239.070673
2022-12-22 22:45:00    239.050327
2022-12-22 23:00:00    239.032494
2022-12-22 23:15:00    239.016260
2022-12-22 23:30:00           NaN
Freq: 15T, dtype: float64
